In [41]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [0]:
import re
import numpy as np
import pandas as pd
from keras.models import Model
from sklearn.utils import shuffle
from string import digits, punctuation
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense

In [43]:
data = pd.read_csv('/content/drive/My Drive/data.csv')
data.head()

,source,english_sentence,hindi_sentence
0,ted,politicians do not have permission to do what ...,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह कर..."
1,ted,"I'd like to tell you about one such child,",मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहू...
2,indic2012,This percentage is even greater than the perce...,यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।
3,ted,what we really mean is that they're bad at not...,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
4,indic2012,.The ending portion of these Vedas is called U...,इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता है।


In [0]:
data = data[data['source'] == 'ted']
data = data[~pd.isnull(data['english_sentence'])]
data.drop_duplicates(inplace=True)

In [0]:
data = data.sample(n=25000, random_state=42)

In [0]:
# convert to lower case
data['english_sentence'] = data['english_sentence'].apply(lambda x : x.lower())
data['hindi_sentence'] = data['hindi_sentence'].apply(lambda x : x.lower())

# remove quotes
data['english_sentence'] = data['english_sentence'].apply(lambda x: re.sub("'", '', x))
data['hindi_sentence'] = data['hindi_sentence'].apply(lambda x: re.sub("'", '', x))

# remove special characters
puncs = set(punctuation)
data['english_sentence']=data['english_sentence'].apply(lambda x: ''.join(c for c in x if c not in puncs))
data['hindi_sentence']=data['hindi_sentence'].apply(lambda x: ''.join(c for c in x if c not in puncs))

# remove all numbers from text
remove_digits = str.maketrans('', '', digits)
data['english_sentence'] = data['english_sentence'].apply(lambda x: x.translate(remove_digits))
data['hindi_sentence'] = data['hindi_sentence'].apply(lambda x: x.translate(remove_digits))
data['hindi_sentence'] = data['hindi_sentence'].apply(lambda x: re.sub("[२३०८१५७९४६|]", "", x))

# remove extra spaces
data['english_sentence'] = data['english_sentence'].apply(lambda x: x.strip())
data['hindi_sentence'] = data['hindi_sentence'].apply(lambda x: x.strip())
data['english_sentence'] = data['english_sentence'].apply(lambda x: re.sub(" +", " ", x))
data['hindi_sentence'] = data['hindi_sentence'].apply(lambda x: re.sub(" +", " ", x))

In [0]:
data['hindi_sentence'] = data['hindi_sentence'].apply(lambda x : '<S> '+ x + ' <E>')

In [0]:
# get all unique english and hindi words

all_eng_words=set()
for eng in data['english_sentence']:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

all_hindi_words=set()
for hin in data['hindi_sentence']:
    for word in hin.split():
        if word not in all_hindi_words:
            all_hindi_words.add(word)

In [0]:
# find out number of words in each sentence

data['length_eng_sentence'] = data['english_sentence'].apply(lambda x:len(x.split(" ")))
data['length_hin_sentence'] = data['hindi_sentence'].apply(lambda x:len(x.split(" ")))

In [0]:
# keep only those sentences whose length <= 20

data = data[data['length_eng_sentence']<=20]
data = data[data['length_hin_sentence']<=20]

In [0]:
max_len_hin = max(data['length_hin_sentence'])
max_len_eng = max(data['length_eng_sentence'])

In [0]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_hindi_words))

num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_hindi_words) + 1 # 1 for padding

In [0]:
input_word2index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_word2index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [0]:
input_index2word = dict((i, word) for word, i in input_word2index.items())
target_index2word = dict((i, word) for word, i in target_word2index.items())

In [0]:
# generate test and train data

data = shuffle(data)

x, y = data['english_sentence'], data['hindi_sentence']

train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2, random_state=42)

In [0]:
def generate_batch(x, y, batch_size = 128):
    while True:
        for j in range(0, len(x), batch_size):
            encoder_input_data = np.zeros((batch_size, max_len_eng),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_len_hin),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_len_hin, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(x[j:j + batch_size], y[j: j + batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_word2index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t < len(target_text.split()) - 1:
                        decoder_input_data[i, t] = target_word2index[word] # decoder input seq
                    if t > 0:
                        decoder_target_data[i, t - 1, target_word2index[word]] = 1.
                        
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [0]:
# encoder

encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, 300, mask_zero=True)(encoder_inputs)
encoder_lstm = LSTM(300, return_state=True)
encoder_outputs, h, c = encoder_lstm(enc_emb)
encoder_states = [h, c]

In [0]:
# decoder

decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, 300, mask_zero=True)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(300, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [0]:
# define the model

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [63]:
# train the model

train_samples = len(train_x)
test_samples = len(test_x)

model.fit_generator(
    generator = generate_batch(train_x, train_y),
    steps_per_epoch = train_samples // 128,
    epochs = 100,
    validation_data = generate_batch(test_x, test_y),
    validation_steps = test_samples // 128
)

Epoch 1/100
154/154 [==============================] - 41s 266ms/step - loss: 6.4263 - val_loss: 6.1140
Epoch 2/100
154/154 [==============================] - 38s 250ms/step - loss: 5.8183 - val_loss: 5.8445
Epoch 3/100
154/154 [==============================] - 39s 253ms/step - loss: 5.4657 - val_loss: 5.5872
Epoch 4/100
154/154 [==============================] - 39s 252ms/step - loss: 5.2175 - val_loss: 5.4825
Epoch 5/100
154/154 [==============================] - 39s 254ms/step - loss: 5.0103 - val_loss: 5.4156
Epoch 6/100
154/154 [==============================] - 39s 252ms/step - loss: 4.8236 - val_loss: 5.3604
Epoch 7/100
154/154 [==============================] - 39s 253ms/step - loss: 4.6541 - val_loss: 5.3254
Epoch 8/100
154/154 [==============================] - 39s 251ms/step - loss: 4.4925 - val_loss: 5.2900
Epoch 9/100
154/154 [==============================] - 39s 253ms/step - loss: 4.3397 - val_loss: 5.2749
Epoch 10/100
154/154 [==============================] - 39s 254m

In [0]:
# encoder model

encoder_model = Model(encoder_inputs, encoder_states)

In [0]:
# decoder model

decoder_state_input_h = Input(shape=(300,))
decoder_state_input_c = Input(shape=(300,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2
)

In [0]:
def predict(input_seq):
    states_value = encoder_model.predict(input_seq)

    target_seq = np.zeros((1,1))
    target_seq[0, 0] = target_word2index['<S>']

    sentence = ''
    while True:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        index = np.argmax(output_tokens[0, -1, :])
        word = target_index2word[index]
        
        if word == '<E>' or len(sentence) > 50:
            break

        sentence += ' ' + word

        target_seq = np.zeros((1,1))
        target_seq[0, 0] = index
        states_value = [h, c]

    return sentence

In [0]:
train_gen = generate_batch(train_x, train_y, batch_size = 1)

In [81]:
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = predict(input_seq)

print('Input English sentence:', train_x[0:1].values[0])
print('Actual Hindi Translation:', train_y[0:1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-3])

Input English sentence: “well yeah yeah thats part of the project”
Actual Hindi Translation: ी हाँ बराबर बराबर यह इस परियोजना का हिस्सा है”
Predicted Hindi Translation:  “जी हाँ बराबर बराबर यह इस परियोजना का हिस्सा 
